In [1]:
#import section
import pandas as pd
import numpy as np
import os
from datetime import datetime
import hkvsobekpy as hkv
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
plt.style.use('ggplot')
plt.style.use('seaborn-poster') 

%matplotlib inline

In [2]:
# # define function
# def readGem(sbkDir, sbkFle, sbkLoc, dteFrom, dteTo):
#     """
#     Read parameter from sobek outputfile

#     Parameters
#     ----------
#     sbkDir : sobek directory
#     """
    
#     rdFlow = hkv.read_his.ReadMetadata(os.path.join(sbkDir, sbkFle))
#     locaties = np.asarray(rdFlow.GetLocations())
#     tijdstappen = rdFlow.GetTimestamps()
#     parameters = rdFlow.GetParameters()
        
#     uitlezen = (np.intersect1d(locaties, sbkLoc)).tolist()
#     df_Hour = rdFlow.MultiWaardenArray(uitlezen,parameters,startMMdd=(1, 1),endMMdd=(12, 31),jaarmax_as='none',drop_lege_jaren=False)
#     df_Hour = df_Hour.loc[dteFrom:dteTo, :] 
        
#     return df_Hour

In [3]:
from_date = '2016-03-31'
from_date = datetime.strptime(from_date,'%Y-%m-%d')
to_date   = '2016-11-1'
to_date = datetime.strptime(to_date, '%Y-%m-%d')

In [4]:
sobek_path = r'P:\PR\3777.10\Werk\notebooks\data\HHSKRAIN.lit\9'
output_path = r'D:\jupyter notebooks\3777.10 WSA Schieland\test'
lstColor = ['C5','C4','C3','C2','C1']

In [5]:
# init reading his structure (time consuming, try to execute this part only once)
pump_his = hkv.read_his.ReadMetadata(os.path.join(sobek_path, 'PUMP.HIS'))

In [6]:
# structure is known, parse in once to dataframe (this is quick)
df_pump = pump_his.DataFrame()
locs = pump_his.KrijgLokaties()

C:\ProgramData\Miniconda3\lib\site-packages\hkvsobekpy\io\his.py:351: UserWarning: this function will deprecate in the future, use function hkvsobekpy.GetLocations
  warnings.warn("this function will deprecate in the future, use function hkvsobekpy.GetLocations")


In [7]:
file = r'D:\jupyter notebooks\3777.10 WSA Schieland\PR3777.10 Waterbalans\conf\Overzicht_gemalen.csv'
df_data = pd.read_csv(file, header=0)

#df_select = df_data[(df_data['Code'] == 'KGM-15')]
for locid, df_select in df_data.groupby('Code') :
    Gemaal = df_select.iloc[0]['Code']
    Gebied =  df_select.iloc[0]['Gemaal']
    Gemaal_GB =['{}'.format(Gemaal), '{}p1'.format(Gemaal), '{}p2'.format(Gemaal), '{}p3'.format(Gemaal), '{}p4'.format(Gemaal)] 
    Inlaat_GB =['f{}'.format(Gemaal), 'i{}'.format(Gemaal), '{}_i'.format(Gemaal)]
    
    locs_pump_GB = (np.intersect1d(locs, Gemaal_GB)).tolist()    
    locs_inlt_GB = (np.intersect1d(locs, Inlaat_GB)).tolist()
    
    # parse dataframe locaties `Gemaal_GB`
    df_gemaal_gb_alldates = df_pump.loc[:, pd.IndexSlice[:, locs_pump_GB]]
    df_pump_GB6 = df_gemaal_gb_alldates[from_date:to_date]   
    
    # parse dataframe locaties `Inlaat_GB`
    df_inlaat_gb_alldates = df_pump.loc[:, pd.IndexSlice[:, locs_inlt_GB]]
    df_inlt_GB6 = df_inlaat_gb_alldates[from_date:to_date]

## Afvoer (gemaal)

In [39]:
file = r'D:\jupyter notebooks\3777.10 WSA Schieland\PR3777.10 Waterbalans\conf\Overzicht_gemalen.csv'
df_data = pd.read_csv(file, header=0)

#df_select = df_data[(df_data['Code'] == 'KGM-15')]
for locid, df_select in df_data.groupby('Code') :
    Gemaal = df_select.iloc[0]['Code']
    Gebied =  df_select.iloc[0]['Gemaal']
    Gemaal_GB =['{}'.format(Gemaal), '{}p1'.format(Gemaal), '{}p2'.format(Gemaal), '{}p3'.format(Gemaal), '{}p4'.format(Gemaal)] 
    Inlaat_GB =['f{}'.format(Gemaal), 'i{}'.format(Gemaal), '{}_i'.format(Gemaal)]
    
    locs_pump_GB = (np.intersect1d(locs, Gemaal_GB)).tolist()    
    locs_inlt_GB = (np.intersect1d(locs, Inlaat_GB)).tolist()
    
    # parse dataframe locaties `Gemaal_GB`
    df_gemaal_gb_alldates = df_pump.loc[:, pd.IndexSlice[:, locs_pump_GB]]
    df_pump_GB6 = df_gemaal_gb_alldates[from_date:to_date]   
    
    # parse dataframe locaties `Inlaat_GB`
    df_inlaat_gb_alldates = df_pump.loc[:, pd.IndexSlice[:, locs_inlt_GB]]
    df_inlt_GB6 = df_inlaat_gb_alldates[from_date:to_date]
    
    #figuur waterstand polder
    fig1 = plt.figure(figsize = (15,8))
    #waterstand polder
    plt.plot(df_pump_GB6.index.values, df_pump_GB6[('Suction-Side Level (m AD)', df_select.iloc[0]['Code'])], color=lstColor[4], label = 'ws polder')
    #streefpeil polder
    df_dum = df_pump_GB6[('Suction-Side Level (m AD)', df_select.iloc[0]['Code'])]*0 +  df_select.iloc[0]['St_peil']
    plt.plot(df_dum.index.values, df_dum[:], label = 'streefpeil polder', color=lstColor[1])
    #in- en uitslagpeil polder    
    i_pump=-1
    for idx, row in df_select.iterrows():
        i_pump +=1
        df_dum = df_pump_GB6[('Suction-Side Level (m AD)', row['Code'])]*0 +  row['In_peil']
        plt.plot(df_dum.index.values, df_dum[:], color=lstColor[i_pump], linestyle=':', label = 'inslag{}'.format(row['Cd_pomp']))
        df_dum = df_pump_GB6[('Suction-Side Level (m AD)', row['Code'])]*0 +  row['Uit_peil']
        plt.plot(df_dum.index.values, df_dum[:], color=lstColor[i_pump], linestyle='--', label = 'uitslag{}'.format(row['Cd_pomp']))
    #opmaak grafiek
    grfTitle = 'Gemaal %s %s-%s waterstand'%(df_select.iloc[0]['Gemaal'],str(from_date)[0:4],str(to_date)[0:4])
    plt.title(grfTitle, fontsize = 20)
    plt.ylabel('m+NAP', fontsize = 18)

    # ylim is ±0.5m NAP streefpeil 
    plt.ylim(df_dum.mean() - 0.5, df_dum.mean() + 0.5)

    leg = plt.legend(fontsize = 16,frameon=True)
    #leg.set_title(title='overloopdebiet', prop = {'size':'xx-large'})
    frame = leg.get_frame()
    frame.set_color('white')

    #plt.legend(loc = 'best', fontsize = 16)
    plt.savefig(os.path.join(output_path, grfTitle))
    fig1.clear()
    plt.close()
    #plt.show()   
    
    #figuur waterstand boezem
    fig2 = plt.figure(figsize = (15,8))
    plt.plot(df_pump_GB6.index.values, df_pump_GB6[('Delivery-Side Level (m AD)',df_select.iloc[0]['Code'])], color=lstColor[4], label = 'ws boezem')
    #streefpeil boezem
    df_dum = df_pump_GB6[('Suction-Side Level (m AD)', df_select.iloc[0]['Code'])]*0 +  df_select.iloc[0]['Bo_peil']
    plt.plot(df_dum.index.values, df_dum[:], label = 'streefpeil boezem', color=lstColor[1])
    #opmaak grafiek
    grfTitle = 'Gemaal %s %s-%s boezem'%(df_select.iloc[0]['Gemaal'],str(from_date)[0:4],str(to_date)[0:4])
    plt.title(grfTitle, fontsize = 20)
    plt.ylabel('m+NAP', fontsize = 18)

    # ylim is ±0.5m NAP streefpeil 
    plt.ylim(df_dum.mean() - 0.5, df_dum.mean() + 0.5)

    #plt.legend(loc = 'best', fontsize = 16)
    leg = plt.legend(fontsize = 16,frameon=True)
    #leg.set_title(title='overloopdebiet', prop = {'size':'xx-large'})
    frame = leg.get_frame()
    frame.set_color('white')
    plt.savefig(os.path.join(output_path, grfTitle))
    fig2.clear() 
    plt.close()
    #plt.tight_layout()
    #plt.show()    
    
    #figuur afvoer polder
    fig3 = plt.figure(figsize = (15,8))
    plt.plot(df_pump_GB6.index.values, df_pump_GB6[('Pump Discharge (m3/s)')].sum(axis=1), label = 'Afvoer', color=lstColor[4])
    #pompcapacitiet polder
    cp_pump = 0
    for idx, row in df_select.iterrows():
        cp_pump += row['Cp_pomp']
        df_dum = df_pump_GB6[('Pump Discharge (m3/s)')].sum(axis=1)*0 + cp_pump
        plt.plot(df_dum.index.values, df_dum[:], label = 'capaciteit {}'.format(row['Cd_pomp']), linestyle=':')
    #opmaak grafiek
    grfTitle = 'Gemaal %s %s-%s afvoer'%(df_select.iloc[0]['Gemaal'],str(from_date)[0:4],str(to_date)[0:4])
    plt.title(grfTitle, fontsize = 20)
    plt.ylabel('m3/s', fontsize = 18)
    leg = plt.legend(fontsize = 16,frameon=True)
    #leg.set_title(title='overloopdebiet', prop = {'size':'xx-large'})
    frame = leg.get_frame()
    frame.set_color('white')
    #plt.legend(loc = 'best', fontsize = 16)
    plt.savefig(os.path.join(output_path, grfTitle))
    fig3.clear()
    plt.close()
    #plt.show()
    
     #figuur inlaat polder
    fig4 = plt.figure(figsize = (15,8))
    plt.plot(df_inlt_GB6.index.values, df_inlt_GB6[('Pump Discharge (m3/s)')].sum(axis=1), label = 'Inlaat', color=lstColor[4])
    #pompcapacitiet polder
    cp_inlt = 0
    for idx, row in df_select.iterrows():
        cp_inlt += row['Cp_inlt']
        df_dum = df_inlt_GB6[('Pump Discharge (m3/s)')].sum(axis=1)*0 + cp_inlt
        plt.plot(df_dum.index.values, df_dum[:], label = 'capaciteit {}'.format(row['Cd_pomp']), linestyle=':')
    #opmaak grafiek
    grfTitle = 'Gemaal %s %s-%s inlaat'%(df_select.iloc[0]['Gemaal'],str(from_date)[0:4],str(to_date)[0:4])
    plt.title(grfTitle, fontsize = 20)
    plt.ylabel('m3/s', fontsize = 18)
    leg = plt.legend(fontsize = 16,frameon=True)
    #leg.set_title(title='overloopdebiet', prop = {'size':'xx-large'})
    frame = leg.get_frame()
    frame.set_color('white')
    plt.savefig(os.path.join(output_path, grfTitle))  
    fig4.clear()
    plt.close()